**Import needed packages/modules**

In [ ]:
# Cell 1
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Rectangle
from scipy.ndimage import convolve, generate_binary_structure

**Define a function to enforce Neumann boundary conditions**
1. If the walls are `conductors`, the edges must have a zero gradient $(\nabla=0)$
2. We can force $\nabla=0$ by ensuring the outermost grid cells have\
the <u>same</u> <mark>potential</mark> as their next adjacent inner grid cells
3. Note that $\nabla=0$ does mean the edges must be at zero *voltage*
4. Carl Neumann (1832-1925)

In [ ]:
# Cell 2
def conductor_edges(a):
    # Carl Neumann (1832-1925)
    # A conductor forces the edges to
    # have zero potential (gradient=0)
    a[0, :] = a[1, :]
    a[-1, :] = a[-2, :]
    a[:, 0] = a[:, 1]
    a[:, -1] = a[:, -2]
    return a

**Define a function to enforce Dirichlet boundary conditions**
1. If the walls are `insulators`, the edges must have zero potential $(V=0)$
2. We can force $V=0$ by setting the outermost edges in the array to be zero
3. Note that $V=0$ does not mean $\nabla=0$ at the walls
4. Johann Dirichlet (1805-1859)

In [ ]:
# Cell 3
def insulator_edges(a):
    # Johann Dirichlet (1805-1859)
    # An insulator forces the edges to
    # have a fixed charge (voltage=0)
    a[0, :] = 0
    a[-1, :] = 0
    a[:, 0] = 0
    a[:, -1] = 0
    return a

**Define a function to solve the Laplacian 2nd order PDE ${\nabla}^2\phi=0$ for electrostatic fields**
1. The function receives a matplotlib axes object in which to render the plot
2. The fucntion receives a **function** to enforce the boundary conditions
3. The function also receives the voltage of the left and right plates

In [ ]:
# Cell 4
def solve_laplace(ax, boundary_func, left_volts, right_volts):
  N = 100 # Number of 2D grid cells in the x & y directions
  grid = np.zeros((N, N))
  grid[30:70, 29:30] = left_volts
  grid[30:70, 70:71] = right_volts
  # Create masks for plates using boolean indexing
  mask_left = grid == left_volts
  mask_right = grid == right_volts
  # Create a convolution kernel to apply over grid each iteration
  kern = generate_binary_structure(2, 1).astype(float) / 4
  kern[1, 1] = 0
  for _ in range(5000): # Number of Jacobi relaxation iterations
      # Create a new grid by using convolution to average
      # every four neighbor cells in the current grid
      grid_next = convolve(grid, kern, mode="constant")
      # Reapply the boundary conditions
      grid_next = boundary_func(grid_next)
      # Reapply the plate voltages
      grid_next[mask_left] = left_volts
      grid_next[mask_right] = right_volts
      # The "next" grid now becomes the current grid
      grid = grid_next
  # Render a colored contour plot of the electrostatic field potential
  surf = ax.contourf(range(N), range(N), grid, cmap="rainbow", levels=20)
  ax.get_figure().colorbar(surf, ax=ax, shrink=0.5)
  # Blacken the two parallel plates
  ax.add_patch(Rectangle((29, 30), 1, 40, edgecolor="k", facecolor="k"))
  ax.add_patch(Rectangle((70, 30), 1, 40, edgecolor="k", facecolor="k"))
  # Title the graph
  if boundary_func == conductor_edges:
      ax.set_title("Conductor Edges")
  else:
      ax.set_title("Insulator Edges")
  ax.set_aspect("equal")

**Define a function to display the electrocstatic field between two parallel plates**
1. The function receives the requested left and right plate voltages
2. The function will display the results of the field between the two plates
3. The situations where the walls are conductors and insulators are both shown

Simulate LEFT plate $=-1V$ and RIGHT plate $=+1V$

In [ ]:
# Cell 5
def plot_plates(left_volts, right_volts):
  plt.figure(figsize=(10, 5.5))
  ax1 = plt.subplot(1, 2, 1)
  ax2 = plt.subplot(1, 2, 2)
  solve_laplace(ax1, conductor_edges, left_volts, right_volts)
  solve_laplace(ax2, insulator_edges, left_volts, right_volts)
  plt.tight_layout()
  plt.show()

# Show the results where left plate is -1V and right plate is +1V
plot_plates(-1, 1)

**Simulate LEFT plate $=-1V$ and RIGHT plate $=$<font color="red">$+9V$</font>**

In [ ]:
# Cell 6
plot_plates(-1, 9)

**Simulate LEFT plate $=$<font color="red">$+9V$</font> and RIGHT plate $=$<font color="red">$+9V$</font>**

In [ ]:
# Cell 7
plot_plates(9, 9)